In [ ]:
import warnings
# warnings.filterwarnings('ignore')

In [2]:
from dataloader import training_dataset, testing_dataset
import torch
from torch.utils.data import DataLoader
from architecture import pipe
from torch import nn
from tqdm import tqdm
import os

D:\ml\mtest\learn_env\lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\ml\mtest\learn_env\lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\ml\mtest\learn_env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
training_data = DataLoader(training_dataset, batch_size=32, shuffle=True)
testing_data = DataLoader(testing_dataset, batch_size=32, shuffle=True)

In [5]:
vision_model = pipe.get_model()
vae = pipe.get_vae()

In [6]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(vision_model.parameters(), lr=1e-7)

In [7]:
state = {'loss': [], 'epochs': [], 'test_loss': []}
def train_loop(dataloader, testing_data, model, loss_fn, optimizer, epochs=10, save_path='./saves'):
    global state
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    all_exps = [int(elm.replace('exp_', '')) if elm != '.ipynb_checkpoints' else -1 for elm in os.listdir(save_path)]

    current_num = max(all_exps) if (len(all_exps) > 0) else 1
    save_path = save_path + '/exp_' + str(current_num + 1)
    os.mkdir(save_path)

    model.to(device)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        model.train()
        loop = tqdm(dataloader, total=len(dataloader), leave=True)
        total_loss = 0

        for batch_idx, (data, targets) in enumerate(loop):
            data = data.to(device)
            targets = targets.to(device)
            
            with torch.no_grad():
                features = vae.encode(data).latent_dist.sample()
            
            # Forward pass
            scores = model(features)
            loss = loss_fn(scores, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update progress bar
            loop.set_postfix(loss=loss.item())

        testing_loss = 0
        for test in tqdm(testing_data):
            X, y = test
            with torch.no_grad():
                model.eval()

                score = model(X)
                testing_loss =+ loss_fn(score, y)

                model.train()

        print(f"Epoch {epoch + 1} average loss: {total_loss / len(dataloader)} with testing loss of {testing_loss / len(testing_data)}")

        state['loss'].append(total_loss / len(dataloader))
        state['epochs'].append(epoch + 1)
        state['test_loss'].append((testing_loss / len(testing_data)).cpu())

        # Save the models after each epoch
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss / len(dataloader),
            'testing_loss': (testing_loss / len(testing_data)).cpu()
        }, f"{save_path}/model_epoch_{epoch + 1}.pth")

    print("Training complete!")

In [ ]:
train_loop(training_data, testing_data, vision_model, loss_fn, optimizer, epochs=10)

Epoch 1/10


  0%|          | 0/108 [00:00<?, ?it/s]D:\ml\mtest\learn_env\lib\site-packages\diffusers\models\lora.py:358: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 6.75 GiB. GPU  (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:924.)
  return F.conv2d(
D:\ml\mtest\learn_env\lib\site-packages\diffusers\models\attention_processor.py:1259: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  hidden_states = F.scaled_dot_product_attention(
  6%|▌         | 6/108 [01:45<20:55, 12.31s/it, loss=0.704]  

In [ ]:
state.keys()